In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
# from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# 1. 문서 로드

In [3]:
def pdf_loader(file_path_list):
    result = []
    for i, file_path in enumerate(file_path_list):
        print(f"{i + 1}/{len(file_path_list)} 문서 로드중")
        loader = PyPDFLoader(file_path)
        docs = loader.load()
        print(f"총 {len(docs)}장의 문서 로드")
        result.extend(docs)

    return result

In [4]:
file_path_list = ['../data/2025년+건강검진사업안내.pdf']

docs = pdf_loader(file_path_list)

1/1 문서 로드중
총 181장의 문서 로드


In [24]:
print(docs[10].page_content)

Chapter 1 건강검진사업 현황
 • 7
구  분 2023예산 2024예산
(a)
2025예산
(b)
증감
(b­a) (%)
합   계 ( A + B ) 858,214 902,012 1,047,892 145,880 16.2
◦ 일반건강검진 사업 773,479 826,364 960,817 134,453 16.3
◦ 영유아건강검진 사업 82,387 73,450 85,237 11,787 16
◦ 영유아 발달 정밀검사비 지원사업 836 786 595 △191 △24.3
◦ 의료급여수급권자 건강검진 사업운영비 254 254 254 - -
◦ 건강검진사업 운영 1,258 1,158 989 △169 △14.6
국   고 ( A ) 9,586 9,466 8,978 △488 △5.2
◦ 일반건강검진 사업
(의료급여생애전환기검진 사업 포함) 6,621 6,621 6,621 - -
◦ 영유아건강검진 사업 617 647 519 △128 △19.8
◦ 영유아 발달 정밀검사비 지원사업 836 786 595 △191 △24.3
◦ 의료급여수급권자 건강검진 사업운영비 254 254 254 - -
◦ 건강검진사업 운영 1,258 1,158 989 △169 △14.6
건강보험재정(B) 848,628 892,546 1,038,914 146,368 16.4
◦ 일반건강검진 사업 766,858 819,743 954,196 134,453 16.4
◦ 영유아건강검진 사업 81,770 72,803 84,718 11,915 16.4
(단위：백만원)
* 국고(A)：의료급여수급권자 검진 관련 비용(국비 기준)
* 건강보험재정(B)：건강보험가입자 검진 관련 비용
Chapter
2025년  건강검진사업  주요  예산1­4


# 2. 텍스트 스플리터(청킹)

In [5]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)

chunks = splitter.split_documents(docs)
print(len(chunks), "전체 잘린 chunk 사이즈")

235 전체 잘린 chunk 사이즈


In [6]:
print(chunks[10].page_content)

Chapter 1 건강검진사업 현황
 • 7
구  분 2023예산 2024예산
(a)
2025예산
(b)
증감
(b­a) (%)
합   계 ( A + B ) 858,214 902,012 1,047,892 145,880 16.2
◦ 일반건강검진 사업 773,479 826,364 960,817 134,453 16.3
◦ 영유아건강검진 사업 82,387 73,450 85,237 11,787 16
◦ 영유아 발달 정밀검사비 지원사업 836 786 595 △191 △24.3
◦ 의료급여수급권자 건강검진 사업운영비 254 254 254 - -
◦ 건강검진사업 운영 1,258 1,158 989 △169 △14.6
국   고 ( A ) 9,586 9,466 8,978 △488 △5.2
◦ 일반건강검진 사업
(의료급여생애전환기검진 사업 포함) 6,621 6,621 6,621 - -
◦ 영유아건강검진 사업 617 647 519 △128 △19.8
◦ 영유아 발달 정밀검사비 지원사업 836 786 595 △191 △24.3
◦ 의료급여수급권자 건강검진 사업운영비 254 254 254 - -
◦ 건강검진사업 운영 1,258 1,158 989 △169 △14.6
건강보험재정(B) 848,628 892,546 1,038,914 146,368 16.4
◦ 일반건강검진 사업 766,858 819,743 954,196 134,453 16.4
◦ 영유아건강검진 사업 81,770 72,803 84,718 11,915 16.4
(단위：백만원)
* 국고(A)：의료급여수급권자 검진 관련 비용(국비 기준)
* 건강보험재정(B)：건강보험가입자 검진 관련 비용
Chapter
2025년  건강검진사업  주요  예산1­4


# 3. 임베딩 생성 chromadb 저장

In [ ]:
db_path = "../07_vectorstore/chromadb_rag_helthcheck"

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory=db_path,
    collection_name="samsung_2024"
)
print("벡터 저장소 저장 완료")

벡터 저장소 저장 완료


# 4. 검색기 구성 (retriever)

In [8]:
retriever = vectorstore.as_retriever()

# 5. 기본 체인 만들기

In [28]:
# 1. 프롬프트
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
     주어진 컨텍스트만 근거로 간결하고 정확하게 대답해라
     컨텍스트에 없으면 문서에 "근거 없음" 이라고 말해라
     
     [출처 작성 규칙]
     - 대답에는 어떤 문서의 몇페이지를 근거하고 있는지 리스트로 작성하라
     - 대답의 부분마다 어떤 문서의 몇페이지를 근거하고 있는지 괄호로 작성하라

     [컨텍스트]
     {context}
     """),
    ("user", "{question}"),
])

# 2. 모델 선택
model = ChatOpenAI(
    temperature=0,
    model = "gpt-4.1-mini",
    verbose=True
)

# 3. output parser 선택
outputparser = StrOutputParser()

# 4. chain 생성
chain = rag_prompt | model | outputparser


# 6. RAG 체인 만들기

In [29]:
# 문서를 합치는 함수
def format_docs(docs):
    return "\n\---\n\n".join([f"메타데이터: {doc.metadata}\n 내용: " + doc.page_content for doc in docs])

In [30]:
rag_chain = (
    {
        "context" : retriever | RunnableLambda(format_docs),
        "question" : RunnablePassthrough()
    }
    | chain
)
rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001C38ABEADD0>, search_kwargs={})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n     주어진 컨텍스트만 근거로 간결하고 정확하게 대답해라\n     컨텍스트에 없으면 문서에 "근거 없음" 이라고 말해라\n     \n     [출처 작성 규칙]\n     - 대답에는 어떤 문서의 몇페이지를 근거하고 있는지 리스트로 작성하라\n     - 대답의 부분마다 어떤 문서의 몇페이지를 근거하고 있는지 괄호로 작성하라\n\n     [컨텍스트]\n     {context}\n     '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object a

In [27]:
result = rag_chain.invoke("일반 건강검진 항목에는 어떤것이 있나요?")
print(result)

일반건강검진 항목은 다음과 같습니다:

1. 건강검진 상담료 및 행정비용
   - 문진, 진찰, 상담
   - 키, 몸무게, 비만도, 허리둘레 측정
   - 혈압측정
   - 시력, 청력 측정
   - 장애인 안전 및 편의관리

2. 흉부방사선 촬영

3. 요검사(요단백)

4. 혈액검사
   - 혈색소
   - 공복혈당
   - 총콜레스테롤
   - 고밀도(HDL) 콜레스테롤
   - 중성지방
   - 저밀도(LDL) 콜레스테롤
   - AST(SGOT)
   - ALT(SGPT)
   - 감마지티피(γ-GTP)
   - 혈청 크레아티닌 검사
   - 신사구체여과율(e-GFR)
   - (단, 콜레스테롤 4종 검사는 남성 24세 이상, 여성 40세 이상에 대해 4년마다)

5. 간염검사
   - B형간염 표면항원 및 항체 (40세 대상)
   - C형간염 항체 (56세 대상)

6. 골밀도 검사 (54, 60, 66세 여성 대상)

7. 인지기능장애 검사 (KDSQ-C 검사 및 상담, 66세 이상 2년마다)

8. 생활습관평가 (40, 50, 60, 70세 대상)

9. 정신건강검사
   - PHQ-9(우울증) 검사 및 상담
   - CAPE-15(조기정신증) 검사 및 상담
   - 대상 연령 및 주기: 20~34세(2년마다), 35~39세(1회), 40~49세(1회), 50~59세(1회), 60~69세(1회), 70~79세(1회)

10. 노인신체기능검사(낙상검사)
    - 하지기능, 평형성 검사 (66, 70, 80세 대상)

[출처: 2025년 건강검진사업안내.pdf, 14~18페이지]


In [31]:
result = rag_chain.invoke("1999년생은 올해 받아야하는 건강검진들만 리스트로 정리해줘")
print(result)

1999년생은 2025년 기준 만 26세입니다.

컨텍스트에 따르면, 일반건강검진 대상자의 검진항목 중 연령별로 시행되는 항목은 다음과 같습니다:

- 콜레스테롤(4종) 검사: 남성 24세 이상, 여성 40세 이상에 4년마다 시행  
- 정신건강검사(PHQ-9, CAPE-15): 20~34세는 2년마다 시행  
- 생활습관평가: 40, 50, 60, 70세에 시행 (26세는 해당 없음)  
- 간염검사: 40세, 56세에 시행 (26세는 해당 없음)  
- 골밀도 검사: 54, 60, 66세 여성 대상 (26세는 해당 없음)  
- 인지기능장애 검사: 66세 이상 2년마다 (26세는 해당 없음)  
- 노인신체기능검사: 66, 70, 80세 대상 (26세는 해당 없음)  

따라서 1999년생(만 26세)은 다음 건강검진을 받아야 합니다:

1. 일반건강검진 기본 항목  
   - 문진, 진찰, 상담  
   - 키, 몸무게, 비만도, 허리둘레  
   - 혈압측정  
   - 시력, 청력 측정  
   - 흉부방사선 촬영  
   - 요검사(요단백)  
   - 혈액검사 (혈색소, 공복혈당, AST, ALT, γ-GTP, 혈청 크레아티닌, e-GFR)  
2. 콜레스테롤(4종) 검사 (남성 24세 이상 대상)  
3. 정신건강검사 (PHQ-9, CAPE-15) - 20~34세 2년마다 시행 대상  

기타 연령별 특화 검진 항목은 해당되지 않습니다.

[출처: 2025년 건강검진 사업안내, 17페이지]
